In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.insert(0,'/Users/jakevogel/git/small_heads/TADPOLE/tadpole/')
import train as tr
from importlib import reload

Using TensorFlow backend.


In [45]:
#reload(tr)

<module 'train' from '/Users/jakevogel/git/small_heads/TADPOLE/tadpole/train.py'>

In [2]:
tr.compute_data_table?

In [4]:
full_df = pd.read_csv('/Users/jakevogel/Science/TADPOLE/inputed50_cleaned.csv')
ref_df = pd.read_csv('/Users/jakevogel/Science/TADPOLE/inputed50_ref.csv')
d1d2 = pd.read_csv('/Users/jakevogel/Science/TADPOLE/TADPOLE_D1_D2.csv')

//anaconda/envs/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,72

In [73]:
full_df.shape

(12741, 374)

In [5]:
full_df.drop(full_df.columns[0],axis=1,inplace=True)

In [75]:
full_df.shape

(12741, 373)

In [6]:
full_df['RID'] = ref_df['RID']
full_df['VISCODE'] = ref_df['VISCODE']
full_df['COLPROT'] = ref_df['COLPROT']
full_df['EXAMDATE'] = ref_df['EXAMDATE']
full_df['Month_bl'] = d1d2['Month_bl']

In [7]:
data = full_df

target = ref_df[['y_DX','y_ADAS13','y_Ventricles_adj_TIV']]
#target = target.dropna(axis=0)
#data = data.loc[target.index]
#data = data.dropna(axis=1)
#target = target.loc[data.index]
target['Month_bl'] = data['Month_bl']
target = target[['Month_bl','y_DX','y_ADAS13','y_Ventricles_adj_TIV']]

//anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
for col in target.columns[1:]:
    data[col] = target[col]

In [10]:

def compute_data_table(sel_index):
    x_ = []
    y_ = []
    rids = []
    selection = data.iloc[sel_index][['RID','Month_bl']]
    for i in range(selection.shape[0]):
        rid, vis_month = selection.iloc[i].values
        #print(rid,vis_month)
        mask = (data[data['RID']==rid]['Month_bl']>vis_month).values

        valid_data = data[data['RID']==rid][mask]
        target_ = target[data['RID']==rid][mask]
        
        #print(mask)
        if valid_data.shape[0]>0:
            # add to x 
            x_.append(data.iloc[sel_index[i]].drop(['VISCODE','EXAMDATE','COLPROT','Month_bl']).values)
            # add to y
            target_ = target_.values
            target_[:,0] = target_[:,0]-vis_month
            #print(target_)
            
            y_.append(target_)
            rids.append(rid)
    return x_,y_,rids
 


In [70]:
data.shape

(12741, 377)

In [53]:
len(rids)
len(x_)

11002

In [33]:
data[data.RID==2]['Month_bl']

0          0.00000
5723       5.86885
5724      35.54100
5725      60.32790
5726      65.67210
5727      72.19670
5728      78.26230
5729      84.42620
5730      90.32790
5731      95.83610
10836    102.03300
11571    108.91800
11959    120.19700
Name: Month_bl, dtype: float64

In [ ]:
108-

In [32]:
data[data.RID==2].index

Int64Index([0, 5723, 5724, 5725, 5726, 5727, 5728, 5729, 5730, 5731, 10836,
            11571, 11959],
           dtype='int64')

In [ ]:
[0, 5291, 5292, 5293, 5294, 5295, 5296, 5297, 5298, 5299, 9706, 10510]

In [11]:
sel_index = data.index
x_,y_, rids = compute_data_table(sel_index) 

In [14]:
y_[0][0]

array([ 5.86885   ,  1.        ,  0.18433181,  1.00739018])

In [2]:
j1,j2,j3 = tr.compute_data_table(False,'/Users/jakevogel/Science/TADPOLE/')

//anaconda/envs/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,72

In [3]:
np.array(j2)[[x for x in range(len(rids)) if rids[x] == 2]][0]

NameError: name 'y_' is not defined

In [13]:
np.array(y_)[[x for x in range(len(rids)) if rids[x] == 2]][0]

array([[  5.86885000e+00,   1.00000000e+00,   1.84331811e-01,
          1.00739018e+00],
       [  3.55410000e+01,   1.00000000e+00,   2.12544402e-01,
          8.63873946e-01],
       [  6.03279000e+01,   1.00000000e+00,   4.69022498e-01,
          9.57802772e-01],
       [  6.56721000e+01,              nan,   3.50432648e-01,
          9.00127795e-01],
       [  7.21967000e+01,   1.00000000e+00,   2.98037101e-01,
          1.05179680e+00],
       [  7.82623000e+01,              nan,   2.94039049e-02,
          9.72556824e-01],
       [  8.44262000e+01,   2.00000000e+00,  -3.00411790e-01,
          9.13991348e-01],
       [  9.03279000e+01,              nan,  -8.72097882e-02,
          1.17986399e+00],
       [  9.58361000e+01,   1.00000000e+00,   4.15590046e-02,
          1.46498029e+00],
       [  1.02033000e+02,              nan,   3.30877771e-02,
          1.22473341e+00],
       [  1.08918000e+02,              nan,   5.35343018e-02,
          1.29214608e+00],
       [  1.20197000e

In [4]:
[x for x in range(len(j3)) if j3[x] == 2]

[0, 5309, 5310, 5311, 5312, 5313, 5314, 5315, 5316, 5317, 9924, 10515]

In [56]:
[x for x in range(len(rids)) if rids[x] == 2]

[0, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413, 10023, 10614]

In [69]:
len(j3)

10898

In [37]:
np.array(y_)[[0, 5291, 5292, 5293, 5294, 5295, 5296, 5297, 5298, 5299, 9706, 10510]][0]

array([[  5.86885000e+00,   1.00000000e+00,   1.84331811e-01,
          1.00739018e+00],
       [  3.55410000e+01,   1.00000000e+00,   2.12544402e-01,
          8.63873946e-01],
       [  6.03279000e+01,   1.00000000e+00,   4.69022498e-01,
          9.57802772e-01],
       [  6.56721000e+01,              nan,   3.50432648e-01,
          9.00127795e-01],
       [  7.21967000e+01,   1.00000000e+00,   2.98037101e-01,
          1.05179680e+00],
       [  7.82623000e+01,              nan,   2.94039049e-02,
          9.72556824e-01],
       [  8.44262000e+01,   2.00000000e+00,  -3.00411790e-01,
          9.13991348e-01],
       [  9.03279000e+01,              nan,  -8.72097882e-02,
          1.17986399e+00],
       [  9.58361000e+01,   1.00000000e+00,   4.15590046e-02,
          1.46498029e+00],
       [  1.02033000e+02,              nan,   3.30877771e-02,
          1.22473341e+00],
       [  1.08918000e+02,              nan,   5.35343018e-02,
          1.29214608e+00],
       [  1.20197000e

In [7]:
np.array(j2)[[x for x in range(len(j3)) if j3[x] == 2]][0]

array([[  5.86885000e+00,   0.00000000e+00,   1.84331811e-01,
          1.00739018e+00],
       [  3.55410000e+01,   0.00000000e+00,   2.12544402e-01,
          8.63873946e-01],
       [  6.03279000e+01,   0.00000000e+00,   4.69022498e-01,
          9.57802772e-01],
       [  6.56721000e+01,   0.00000000e+00,   3.50432648e-01,
          9.00127795e-01],
       [  7.21967000e+01,   0.00000000e+00,   2.98037101e-01,
          1.05179680e+00],
       [  7.82623000e+01,   0.00000000e+00,   2.94039049e-02,
          9.72556824e-01],
       [  8.44262000e+01,   1.00000000e+00,  -3.00411790e-01,
          9.13991348e-01],
       [  9.03279000e+01,   1.00000000e+00,  -8.72097882e-02,
          1.17986399e+00],
       [  9.58361000e+01,   0.00000000e+00,   4.15590046e-02,
          1.46498029e+00],
       [  1.02033000e+02,   0.00000000e+00,   3.30877771e-02,
          1.22473341e+00],
       [  1.08918000e+02,   0.00000000e+00,   5.35343018e-02,
          1.29214608e+00],
       [  1.20197000e

In [44]:
np.array(j2)[[0, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413, 10023, 10614]][0]

array([[  5.77049000e+00,   0.00000000e+00,   1.84331811e-01,
          1.00739018e+00],
       [  1.26230000e+01,   0.00000000e+00,   4.69022498e-01,
          9.57802772e-01],
       [  1.20984000e+01,   0.00000000e+00,   3.50432648e-01,
          9.00127795e-01],
       [  7.60656000e+00,   0.00000000e+00,   2.98037101e-01,
          1.05179680e+00],
       [  1.31148000e+01,   0.00000000e+00,   2.94039049e-02,
          9.72556824e-01],
       [  2.55738000e+00,   1.00000000e+00,  -3.00411790e-01,
          9.13991348e-01],
       [  1.16393000e+01,   1.00000000e+00,  -8.72097882e-02,
          1.17986399e+00],
       [  1.21639000e+01,   0.00000000e+00,   4.15590046e-02,
          1.46498029e+00],
       [  5.67213000e+00,   0.00000000e+00,   3.30877771e-02,
          1.22473341e+00],
       [  9.60000000e+01,   0.00000000e+00,   5.35343018e-02,
          1.29214608e+00],
       [  6.06885000e+01,   0.00000000e+00,   1.27051703e-01,
          1.24907142e+00]])

In [ ]:
j1,j2,j3 = tr.compute_data_table

In [35]:
y_

[array([[  5.86885000e+00,   1.00000000e+00,   1.84331811e-01,
           1.00739018e+00],
        [  3.55410000e+01,   1.00000000e+00,   2.12544402e-01,
           8.63873946e-01],
        [  6.03279000e+01,   1.00000000e+00,   4.69022498e-01,
           9.57802772e-01],
        [  6.56721000e+01,              nan,   3.50432648e-01,
           9.00127795e-01],
        [  7.21967000e+01,   1.00000000e+00,   2.98037101e-01,
           1.05179680e+00],
        [  7.82623000e+01,              nan,   2.94039049e-02,
           9.72556824e-01],
        [  8.44262000e+01,   2.00000000e+00,  -3.00411790e-01,
           9.13991348e-01],
        [  9.03279000e+01,              nan,  -8.72097882e-02,
           1.17986399e+00],
        [  9.58361000e+01,   1.00000000e+00,   4.15590046e-02,
           1.46498029e+00],
        [  1.02033000e+02,              nan,   3.30877771e-02,
           1.22473341e+00],
        [  1.08918000e+02,              nan,   5.35343018e-02,
           1.29214608e+00],

In [ ]:
j

In [33]:
output = tr.parse_data(x_,y_,data[data.RID==2]['RID'])

ValueError: cannot convert float NaN to integer

In [27]:
full_df[full_df.RID==2]['Month_bl'] 

0          0.00000
5723       5.86885
5724      35.54100
5725      60.32790
5726      65.67210
5727      72.19670
5728      78.26230
5729      84.42620
5730      90.32790
5731      95.83610
10836    102.03300
11571    108.91800
11959    120.19700
Name: Month_bl, dtype: float64